In [6]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# prepare dataset

batch_size = 64
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])  # 归一化,均值和方差

train_dataset = datasets.MNIST(
    root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(
    root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# design model using class


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层
        self.conv1 = torch.nn.Conv2d(1, 5, padding=1, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(5, 10, kernel_size=3)
        self.conv3 = torch.nn.Conv2d(10, 15, kernel_size=3)
        # 池化层
        self.pooling = torch.nn.MaxPool2d(2)
        # 全连接层
        self.l1 = torch.nn.Linear(60, 1024)
        self.l2 = torch.nn.Linear(1024, 512)
        self.l5 = torch.nn.Linear(512, 10)

    def forward(self, x):
        conv_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = F.relu(self.pooling(self.conv3(x)))
        x = x.view(conv_size, -1)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l5(x)
        return x  # 最后一层不做激活，不进行非线性变换


model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)

# training cycle forward, backward, update


def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' %
                  (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0


def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # dim = 1 列是第0个维度，行是第1个维度
            _, predicted = torch.max(outputs.data, dim=1)
            # print("predicted = ",predicted,"\tlabel = ",labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))


if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()


[1,   300] loss: 0.869
[1,   600] loss: 0.175
[1,   900] loss: 0.120
accuracy on test set: 97 % 
[2,   300] loss: 0.095
[2,   600] loss: 0.090
[2,   900] loss: 0.086
accuracy on test set: 97 % 
[3,   300] loss: 0.071
[3,   600] loss: 0.070
[3,   900] loss: 0.069
accuracy on test set: 98 % 
[4,   300] loss: 0.055
[4,   600] loss: 0.060
[4,   900] loss: 0.059
accuracy on test set: 98 % 
[5,   300] loss: 0.051
[5,   600] loss: 0.052
[5,   900] loss: 0.048
accuracy on test set: 98 % 
[6,   300] loss: 0.040
[6,   600] loss: 0.044
[6,   900] loss: 0.047
accuracy on test set: 98 % 
[7,   300] loss: 0.034
[7,   600] loss: 0.045
[7,   900] loss: 0.042
accuracy on test set: 97 % 
[8,   300] loss: 0.031
[8,   600] loss: 0.036
[8,   900] loss: 0.036
accuracy on test set: 98 % 
[9,   300] loss: 0.031
[9,   600] loss: 0.034
[9,   900] loss: 0.031
accuracy on test set: 98 % 
[10,   300] loss: 0.029
[10,   600] loss: 0.029
[10,   900] loss: 0.031
accuracy on test set: 98 % 


**ps:计算output维度方式，预先计算一次看output.shape的值**

In [5]:
import torch
in_channels ,out_channels = 1, 10       #输入维度   输出维度
width, height = 28, 28                #图片宽度   图片高度
my_1_kernel_size = 3    #卷积核大小
batch_size = 64 

input = torch.randn(batch_size, in_channels, width, height)
conv_layer = torch.nn.Conv2d(in_channels, out_channels, kernel_size = my_1_kernel_size)
output = conv_layer(input)

print(output.shape) #输出参数： batch_size  ,输出维度   ，输出宽度  ，输出高度  
#下一个全连接层输入长度即：输出维度 * 输出宽度 * 输出高度


torch.Size([1, 10, 98, 98])
